<a href="https://colab.research.google.com/github/2017Hk/Deep_Learning_in_physical_system/blob/main/DL_project_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y libomp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (1,025 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

In [2]:
from google.colab import files
import os

# Upload the file
uploaded = files.upload()

# Define the destination directory and file name
destination_directory = '/content/data'
file_name = 'fem_code.zip'  # Change this to your desired file name

# Move the uploaded file to the destination directory with the desired file name


Saving femcode(DL_executed).zip to femcode(DL_executed).zip


In [11]:
os.makedirs('/content/data/prop', exist_ok=True)

In [8]:
target_directory = '/content/'
!unzip -q {/content/femcode(DL_executed).zip} -d {target_directory}

unzip:  cannot find or open /content/data/fem_code.zip, /content/data/fem_code.zip.zip or /content/data/fem_code.zip.ZIP.


In [4]:
import subprocess
import os
import numpy as np
import shutil
import re

In [9]:
# Source and destination file paths
source_path = '/content/femcode(DL_executed)/cont_ver2/prop.dat'
# destination_path = 'path/to/destination/file.txt'

# Copy the file
shutil.copyfile(source_path, os.path.join(target_directory,'prop.dat'))

'/content/prop.dat'

In [6]:
def read_prop_data(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            # Split the line by spaces and convert each value to float or int
            line_data = [float(value) if '.' in value else int(value) for value in line.strip().split()]
            data.append(line_data)
    return data

def write_prop_data(filename, data):
    with open(filename, 'w') as f:
        for line in data:
            f.write(' '.join([str(val) for val in line]) + '\n')

def read_displacement_file(dis_file_path):
    with open(dis_file_path, 'r') as file:
        # Initialize an empty list to store the fourth column data
        column_4_data = []
        # Iterate over each line in the file
        for line in file:
            # Split the line into columns based on whitespace
            columns = line.split()
            # Extract the data from the fourth column (index 3, since Python uses zero-based indexing)
            fourth_column_data = columns[3]
            # Append the fourth column data to the list
            column_4_data.append(fourth_column_data)
    column_4_floats = [float(x) for x in column_4_data]
    displace = np.array(column_4_floats)
    return displace

def write_displacement_data(filename, data):
    with open(filename, 'w') as f:
        for line in data:
            f.write(str(line) + '\n')

def generate_hole(data_in, existing_holes):
    while True:
        row_size = np.random.randint(1, 10)  # Size of hole
        col_size = np.random.randint(1, 10)
        position = np.random.randint(41, 361)  # Position of hole

        position_list = []
        valid_hole = True

        for i in range(col_size):
            for j in range(row_size):
                pose_index = position + j + i * 20
                if pose_index > 381:
                    valid_hole = False
                    break
                position_list.append(pose_index)
            if not valid_hole:
                break

        if not valid_hole:
            continue

        if all((pos not in existing_holes and
                all(abs(pos - existing_hole) > 21 for existing_hole in existing_holes))
               for pos in position_list):
            for pos in position_list:
                data_in[pos] = [1]
            existing_holes.update(position_list)
            break

    return data_in

def get_max_index(output_dir, prefix):
    max_index = -1
    pattern = re.compile(rf"{prefix}\s\((\d+)\)")
    for filename in os.listdir(output_dir):
        match = pattern.search(filename)
        if match:
            index = int(match.group(1))
            if index > max_index:
                max_index = index
    return max_index

In [22]:
# Define the file paths
home_directory = os.path.expanduser('~')
prop_file_path = '/content/femcode(DL_executed)/cont_ver2/prop.dat'
dis_file_path = '/content/femcode(DL_executed)/cont_ver2/displacement'
# dis_file_path = '/content/femcode(DL_executed)/cont_ver2/test_disp'
main_e_path = '/content/femcode(DL_executed)/cont_ver2/main.e'
output_dir = '/content/data/prop'

# Check if the file exists
if os.path.exists(prop_file_path):
    print('File exists.')
else:
    print('File does not exist.')

# Get the maximum existing index
max_index = get_max_index(output_dir, 'prop.dat')
start_index = max_index + 1

# Number of new files to generate
n = 100 # Adjust this value as needed

for i in range(start_index, start_index + n):

    # Source and destination file paths
    source_path = os.path.join(os.path.join(target_directory,'prop.dat'))

    # Copy the file
    shutil.copyfile(source_path, prop_file_path)

    # Paths for storing modified data
    prop_output_path = os.path.join(output_dir, f'prop.dat ({i})')
    dis_output_path = os.path.join(output_dir, f'displacement ({i})')

    # Read and modify prop data
    data = read_prop_data(prop_file_path)

    # Generate two holes with the specified constraints
    existing_holes = set()
    data = generate_hole(data, existing_holes)
    data = generate_hole(data, existing_holes)

    write_prop_data(prop_file_path, data)

    if not os.path.exists(main_e_path):
        raise FileNotFoundError("Main.e file not found at:", main_e_path)

# Check if the file is executable
    if not os.access(main_e_path, os.X_OK):
        raise PermissionError("Main.e file is not executable:", main_e_path)

    # Execute the external program
    result = subprocess.run([main_e_path], check=True)

    if result.returncode != 0:
        print("Error: subprocess failed")
        print("Error:", result.stderr)
    else:
        print("Subprocess ran successfully")

    # Read modified prop data and displacement results
    modified_prop = read_prop_data(prop_file_path)
    modified_dis = read_displacement_file(dis_file_path)

    # Save the modified prop data and displacement results
    write_prop_data(prop_output_path, modified_prop)
    write_displacement_data(dis_output_path, modified_dis)


File exists.


CalledProcessError: Command '['/content/femcode(DL_executed)/cont_ver2/main.e']' returned non-zero exit status 29.

In [13]:
import shutil
shutil.rmtree('/content/data/prop')
os.makedirs('/content/data/prop', exist_ok=True)

In [17]:
# Define a function to extract data
def read_data(filename):

    data = []
    with open(filename, 'r') as f:
        for line in f:

            # Split the line by spaces and convert each value to float
            line_data = [float(value) for value in line.strip().split()]
            data.append(line_data)
    return data

In [18]:
import os

# Set the directory path
directory_path = '/content/data/prop'

# List all items in the directory
items = os.listdir(directory_path)

# Filter out only the files
files = [f for f in items if os.path.isfile(os.path.join(directory_path, f))]

# Count the number of files
number_of_files = len(files)
print(f"Number of files in '{directory_path}': {number_of_files}")


Number of files in '/content/data/prop': 200


In [19]:
# Displacement data set
folder_path = '/content/data/prop'
# Input data
diplacement_list=[]
for i in range(0,number_of_files//2):
    file_path = os.path.join(folder_path, f'displacement ({i})')               # Calling file displacement data
    data=read_data(file_path)



    diplacement_list.append(data)

displacement_array = np.array(diplacement_list)
displacement_array=displacement_array.reshape(displacement_array.shape[0],displacement_array.shape[1])
print(displacement_array.shape)

(100, 1681)


In [20]:
# Property data set
# Labels

prop_list=[]

for i in range(0,number_of_files//2):
    file_path = os.path.join(folder_path, f'prop.dat ({i})')               # Calling file prop.dat data
    data=read_data(file_path)



    prop_list.append(data[1:401])

prop_array=np.array(prop_list)
prop_array=prop_array.reshape(prop_array.shape[0],prop_array.shape[1])

print(prop_array.shape)

(100, 400)


In [21]:
# Calculate variance for each feature
print('variance of the data to under the data')
print('Property data :',prop_array.var())
print('Displacement data :',displacement_array.var())

from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler_prop = MinMaxScaler()
scaler_displacement = MinMaxScaler()


# Fit the scaler to the data and transform it
normalized_prop = scaler_prop.fit_transform(prop_array)
normalized_displacement = scaler_prop.fit_transform(displacement_array)

# Calculate variance for each feature
print('variance of the normalized data to under the data')
print('Property data :',normalized_prop.var())
print('Displacement data :',normalized_displacement.var())

variance of the data to under the data
Property data : 4.5320214842455137e+20
Displacement data : 2.7140488914229156e-08
variance of the normalized data to under the data
Property data : 0.18905273437500003
Displacement data : 0.0


In [11]:
# Define the folder to be zipped
folder_to_zip = '/content/data/prop'

# Define the output zip file path (without .zip extension)
output_filename = '/content/data_2'

# Create a zip archive
shutil.make_archive(output_filename, 'zip', folder_to_zip)

'/content/data_2.zip'